In [ ]:
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
#import tensorflow.compat.v1 as tf
#tf.disable_v2_behavior()
import tensorflow as tf
from keras.layers import Input, LSTM, RepeatVector, Activation,Dense
from keras.models import Model
from tensorflow.keras.optimizers import Adam
import keras.backend as K
import pandas as pd
import statsmodels
from scipy import stats
import pdb

In [ ]:
import numpy as np
import math
import glob

In [ ]:
from random import randint
from numpy import array
from numpy import argmax
import keras.backend as K
from tensorflow.keras import models
from numpy import array_equal
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import TimeDistributed
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Lambda
from tensorflow.keras import backend as K

#from scipy.ndimage.interpolation import shift

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        print(e)

In [ ]:
max_input_sequence= 72
max_output_sequence=72
input_dimension=101
print('max_input_sequence: ', max_input_sequence)
print('max_output_sequence: ', max_output_sequence)

In [ ]:
#data=np.load('Stochastic_Normalize_data_Seq72.npy')
#data=data[:,0:50,0:6]
data=pd.read_csv('CostumeStochasticNormalizedData_Seq72_191F.csv')
data2=np.asarray(data.values)
data2=np.reshape(data2,(10690,72,191))
data3=data2[:,:,0:100]
data4=data2[:,:,190:191]
data2=np.concatenate((data3,data4),axis=2)
data2.shape

In [ ]:
data2[0,0:72,0]

In [ ]:
X_train_padded =data2
y_train_padded=data2
print("X_train_padded shape: ",X_train_padded.shape)
print("y_train_padded shape: ",y_train_padded.shape)

In [ ]:
def one_hot_encode(sequence, n_unique):
    encoding = list()
    for value in sequence:
        vector = [0 for _ in range(n_unique)]
        vector[value] = 1
        encoding.append(vector)
    return array(encoding)

# decode a one hot encoded string
def one_hot_decode(encoded_seq):
    return [argmax(vector) for vector in encoded_seq]

In [ ]:
def shift_elements(arr, num, fill_value):
    result = np.empty_like(arr)
    result[:num] = fill_value
    result[num:] = arr[:-num]
    return result

In [ ]:
encoder_input_data = X_train_padded.copy()
decoder_target_data = y_train_padded.copy()
decoder_input_data = decoder_target_data.copy()
for i, samples in enumerate(decoder_target_data):
    #pdb.set_trace()
    shifted=shift_elements(samples,1,-1)
    #shifted= shift(samples,[1,0], cval=-1) google implimentation
    decoder_input_data[i]=shifted
#decoder_target_data=decoder_input_data.copy() # test if it works better
print("Data for Train")
print('encoder_input_data (X): ', encoder_input_data[1])
print('decoder_input_data (teacher forcing): ',decoder_input_data[1])
print('decoder_target_data (y):',decoder_target_data[1])
print(encoder_input_data.shape)


In [ ]:
LSTMoutputDimension = 301

In [ ]:
encoder_inputs= Input(shape=(max_input_sequence, input_dimension), name='encoder_inputs')
mask_value= [-1]*input_dimension
masking = tf.keras.layers.Masking(mask_value=mask_value)
encoder_inputs_masked = masking(encoder_inputs)
encoder_lstm=LSTM(LSTMoutputDimension, return_state=True, name='encoder_lstm')
LSTM_outputs, state_h, state_c = encoder_lstm(encoder_inputs_masked)
encoder_states = [state_h, state_c]
decoder_inputs = Input(shape=(None, input_dimension), name='decoder_inputs')
decoder_lstm = LSTM(LSTMoutputDimension, return_sequences=True, return_state=True, name='decoder_lstm')
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(input_dimension, activation='linear', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_outputs)

#mask1=[False]*input_dimension
#missing_index=np.arange(1,100,2)
#mask1=np.asarray(mask1,dtype=bool)
#mask1[missing_index]=True

#mask2=~mask1
#decoder_softmax = tf.keras.layers.Softmax()



#soft_decoder_outputs=decoder_softmax(decoder_outputs, mask1)

#new_decoder_outputs = tf.keras.layers.Add()([decoder_outputs,soft_decoder_outputs])

model_encoder_training = Model([encoder_inputs, decoder_inputs],decoder_outputs, name='model_encoder_training')

In [ ]:
#tf.keras.losses.MeanSquaredError()
#loss_object = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
#loss_BCE=tf.keras.losses.BinaryCrossentropy(from_logits=True,reduction=tf.keras.losses.Reduction.NONE)
#loss= loss_object+loss_BCE
def custom_loss(y_true, y_pred):
    
    loss=0
    loss_object = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    #loss_BCE=tf.keras.losses.BinaryCrossentropy(from_logits=False,reduction=tf.keras.losses.Reduction.NONE)
    
    
    #missing_index=[1]#np.arange(1,100,2)
    #all_index=[i for i in range(3) if not i in missing_index]

    loss=loss_object(y_true,y_pred)
    
    #loss1=loss_object(tf.gather(y_true,all_index),tf.gather(y_pred,all_index))
    #loss2=loss_BCE(tf.gather(y_true,missing_index),tf.gather(y_pred,missing_index))
    #loss=tf.reduce_mean(loss1)+tf.reduce_mean(loss2) 
    return loss


opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model_encoder_training.compile(optimizer=opt, loss=custom_loss, metrics=['accuracy'])
model_encoder_training.summary()

In [ ]:
plot_model(model_encoder_training, show_shapes=True)

In [ ]:
decoder_target_data[0,:,:]

In [ ]:
es = EarlyStopping(monitor='val_loss', min_delta=1e-6, mode='min', verbose=1, patience=500,restore_best_weights=True)
model_encoder_training.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=1000,
          epochs=5000,
          validation_split=0.1,callbacks=[es])

In [ ]:
"""
def train_test(model, X_train, y_train , X_test, y_test, epochs=500, batch_size=500, patience=5,verbose=0):

    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)
    print('training for ',epochs,' epochs begins with EarlyStopping(monitor= val_loss, patience=',patience,')....')
    history=model.fit(X_train, y_train, validation_split= 0.1, epochs=epochs,batch_size=batch_size, verbose=verbose, callbacks=[es])
    print(epochs,' epoch training finished...')
    _, train_acc = model.evaluate(X_train, y_train, batch_size=batch_size, verbose=0)
    _, test_acc = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    print('\nPREDICTION ACCURACY (%):')
    print('Train: %.3f, Test: %.3f' % (train_acc*100, test_acc*100))
    # summarize history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(model.name+' accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(model.name+' loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
"""


In [ ]:
"""

train_test(model_encoder_training, [encoder_input_data, decoder_input_data], decoder_target_data ,
           [encoder_input_data, decoder_input_data], 
           decoder_target_data, epochs=20, batch_size=200, patience=2,verbose=2)
"""


In [ ]:
encoder_model = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(LSTMoutputDimension,))
decoder_state_input_c = Input(shape=(LSTMoutputDimension,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [ ]:
def decode_sequence(states_value):
    

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, input_dimension))
    
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0,:] = -1 
    
    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_seq = list()
    while not stop_condition:

        # in a loop
        # decode the input to a token/output prediction + required states for context vector
        #pdb.set_trace()
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # add the predicted token/output to output sequence
        decoded_seq.append(output_tokens[0, -1, :])
        

        # Exit condition: either hit max length
        # or find stop character.
        
      
        if (np.sum(output_tokens[0, -1, :]==[-1]*3)>1 or
           len(decoded_seq) == max_output_sequence or 
           -1 in output_tokens[0, -1,:]):
            stop_condition = True

        # with the predicted token/output 
        target_seq = output_tokens
        # updating
        states_value = [h, c]
     
    decoded_seq=np.asarray(decoded_seq)
    decoded_seq=np.reshape(decoded_seq,(len(decoded_seq),input_dimension))
    return decoded_seq

In [ ]:
correct =0 
sampleNo =  1
for sample in range(0,sampleNo):
    # Encode the input as state vectors
    pdb.set_trace()
    states_value = encoder_model.predict(encoder_input_data[sample].reshape(1,max_input_sequence,input_dimension))
    predicted= decode_sequence(states_value)
    if (decoder_target_data[sample])== predicted+ [-1] * (max_output_sequence- len(predicted)):
        correct+=1
print('Accuracy: ', correct/sampleNo)

In [ ]:
test=np.abs(decoder_target_data[sample]-predicted)
input_dimension

In [ ]:
input_dimension

In [ ]:
1/(1+math.e**2)

In [ ]:
math.tanh(-5)

In [ ]:
math.tanh(-1)